In [1]:
import io
from pathlib import Path
import pickle
from PIL import Image

import numpy as np

import requests
from tqdm.notebook import tqdm

In [2]:
with open('name_to_avatar_map.pickle', 'rb') as f:
    name_to_avatar_map = pickle.load(f)
len(name_to_avatar_map)

629

In [3]:
avdir = Path('avatars/')
default = avdir / 'default'
nondefault = avdir / 'nondefault'
for d in [avdir, default, nondefault]:
    if not d.is_dir():
        d.mkdir()

Test run - first one is a custom, second is a default

In [4]:
nm, avurl = next(iter(name_to_avatar_map.items()))

r1 = requests.get(avurl)
im1 = Image.open(io.BytesIO(r1.content))

arr1 = np.array(im1)
flattenedarr1 = arr1.reshape(arr1.size//arr1.shape[-1], arr1.shape[-1])
unique_colors1 = np.unique(flattenedarr1, axis=0)
unique_colors1.shape[0], r1.headers['Content-Type']

(20560, 'image/jpeg')

In [5]:
nm = 'walkerna22' #this is a random user who has a default avatar
avurl = name_to_avatar_map[nm]

r1 = requests.get(avurl)
im1 = Image.open(io.BytesIO(r1.content))

arr1 = np.array(im1)
flattenedarr1 = arr1.reshape(arr1.size//arr1.shape[-1], arr1.shape[-1])
unique_colors1 = np.unique(flattenedarr1, axis=0)
unique_colors1.shape[0], r1.headers['Content-Type']

(2, 'image/png')

People who specifically requested to be on the list despite having default avatars or things that look like default avatars:

In [6]:
override_list_include = ['larrybradley']
override_list_exclude = ['pre-commit-ci[bot]', 'dependabot[bot]']

In [7]:
for nm, avurl in tqdm(list(name_to_avatar_map.items())):
    r = requests.get(avurl)
    im = Image.open(io.BytesIO(r.content))
    arr = np.array(im)
    flattenedarr = arr.reshape(arr.size//arr.shape[-1], arr.shape[-1])
    unique_colors = np.unique(flattenedarr, axis=0)

    fnm = nm + '.' + r.headers['Content-Type'].split('/')[-1]

    if nm in override_list_exclude:
        continue
        
    if nm not in override_list_include and unique_colors.shape[0] == 2:
        outfn = default / fnm
    else:
        outfn = nondefault / fnm

    with outfn.open('wb') as f:
        f.write(r.content)
    

  0%|          | 0/629 [00:00<?, ?it/s]